In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import t, f, norm

In [14]:
def PSIFunc(sample):
    n = len(sample)
    return sample[:, :-1]

def FFunc(sample):
    PSI = PSIFunc(sample)
    return PSI.T @ PSI

def BetaFunc(sample):
    Y = sample[:, -1]
    PSI = PSIFunc(sample)
    F = FFunc(sample)
    return np.array(np.linalg.inv(F) @ PSI.T @ Y)

def EFunc(sample):
    Y = sample[:, -1]
    PSI = PSIFunc(sample)
    return np.array(Y - PSI @ BetaFunc(sample))

def RSSFunc(sample):
    e = EFunc(sample)
    return e @ e.T

def TSSFunc(sample):
    Y = sample[:, -1]
    mean = np.mean(Y)
    return np.sum((Y - mean) ** 2)

def R2Func(sample):
    tss = TSSFunc(sample)
    rss = RSSFunc(sample)
    return (tss - rss) / tss

def PrintEq(beta):
    print(f"y =", end='')
    for i in range(0, len(beta)):
        print(f"{'+' if beta[i] > 0 else '-'} {abs(round(beta[i], 1))}*x{i} ", end='')
    print('+ e')

In [15]:
Y = [83, 85, 84, 85, 85, 86, 86, 87, 86, 87, 87, 87, 88, 88, 88, 88, 88, 89, 90, 89, 90, 90, 91, 90, 92]
Y = np.array(Y)

In [16]:
counts = np.array([2, 6, 11, 4, 2])
count = np.sum(counts)
sample = np.array([[0] * (len(counts) + 1) ] * count)
line = 0
for i in range(len(counts)):
    for j in range(counts[i]):
        sample[line][i] = 1
        sample[line][-1] = Y[line]
        line += 1

In [17]:
beta = BetaFunc(sample)
PrintEq(beta)

y =+ 84.0*x0 + 85.5*x1 + 87.8*x2 + 90.0*x3 + 91.0*x4 + e


In [7]:
PSIFunc(sample)

array([[1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1]])

In [8]:
R2Func(sample)

0.8106060606060607

In [8]:
def R2PVALUE(sample):
    r2 = R2Func(sample)
    n = len(sample)
    p = len(BetaFunc(sample))
    delta_tss = r2 / (1 - r2) * (n - p) / (p - 1) 
    p_value_tss = f.sf(delta_tss, p - 1, n - p)
    return p_value_tss

In [9]:
p_value_tss = R2PVALUE(sample)
if (p_value_tss <= 0.05):
    print('Влияние факторов существенно, p_value =', round(p_value_tss, 5))

Влияние факторов существенно, p_value = 0.0


In [9]:
def delta(sample, i,  j):
    beta = BetaFunc(sample)
    d = beta[i] - beta[j]
    rss = RSSFunc(sample)
    F_inv = np.linalg.inv(FFunc(sample))
    n = len(sample)
    p = len(beta)
    return d / np.sqrt(rss * (F_inv[i][i] + F_inv[j][j])) * np.sqrt(n - p)

def F(sample):
    beta = BetaFunc(sample)
    f_obrat = np.linalg.inv(FFunc(sample))
    p_vals = []
    n = len(sample)
    p = len(beta)
    for i in range(len(beta)):
        p_vals.append([])
        for j in range(i + 1, len(beta)):
            p_value = 2 * t.sf(abs(delta(sample, i, j)), n - p)
            p_vals[i].append(p_value)
    p_vals.pop(-1)
    return p_vals

In [10]:
p_vals = F(sample)

In [17]:
c = 10
for i in range(len(p_vals)):
    for j in range(len(p_vals[i])):
        print(f'x{i} and x{j + i + 1}', end=' ')
        alp = 0.05 / c
        print(f'p-val = {round(p_vals[i][j], 4)}  alpha = {round(alp,3)}', end=' ')
        if (p_vals[i][j] > alp):
            print('РАВНЫ')
        else:
            print('НЕРАВНЫЕ')
        c -= 1
        print()

x0 and x1 p-val = 0.1031  alpha = 0.005 РАВНЫ

x0 and x2 p-val = 0.0002  alpha = 0.006 НЕРАВНЫЕ

x0 and x3 p-val = 0.0  alpha = 0.006 НЕРАВНЫЕ

x0 and x4 p-val = 0.0  alpha = 0.007 НЕРАВНЫЕ

x1 and x2 p-val = 0.0004  alpha = 0.008 НЕРАВНЫЕ

x1 and x3 p-val = 0.0  alpha = 0.01 НЕРАВНЫЕ

x1 and x4 p-val = 0.0  alpha = 0.013 НЕРАВНЫЕ

x2 and x3 p-val = 0.0024  alpha = 0.017 НЕРАВНЫЕ

x2 and x4 p-val = 0.001  alpha = 0.025 НЕРАВНЫЕ

x3 and x4 p-val = 0.2958  alpha = 0.05 РАВНЫ

